In [ ]:
import math
import os

img_dir = '/home/chanvo/PycharmProjects/datasets/sandtable/images'
lab_dir = '/home/chanvo/PycharmProjects/datasets/sandtable/labels'
imgs = os.listdir(img_dir)

In [ ]:
from utils.general import xywh2xyxy
import numpy as np
from PIL import Image
import math
import matplotlib.pyplot as plt
from matplotlib import patches
from tqdm import tqdm

patch = Image.open('patch.png').convert('RGB')

pbar = tqdm(imgs)
for img_name in pbar:
    lab_name = os.path.splitext(img_name)[0] + '.txt'
    img_path = img_dir + '/' + img_name
    lab_path = lab_dir + '/' + lab_name
    lab = np.loadtxt(lab_path, ndmin=2)

    pbar.set_description(f"{img_name}  {len(lab)}")

    img = Image.open(img_path).convert('RGB')
    w, h = img.size

    with open(lab_path, 'r') as f:
        olab = f.read()
    if len(lab) > 0:
        lab = lab[lab[:, 0] != 4][:, 1:]
        lab[:, 0] *= w
        lab[:, 2] *= w
        lab[:, 1] *= h
        lab[:, 3] *= h
        for box in lab:
            nw = nh = 0.2 * math.sqrt(box[2] * box[3])
            npatch = patch.resize((int(nw), int(nh)))
            olab += '7 %.6f %.6f %.6f %.6f\n' % (box[0] / w, box[1] / h, nw / w, nh / h)
            img.paste(npatch, (int(box[0] - nw / 2), int(box[1] - nh / 2)))
    img.save('patch_dataset/images/' + img_name)
    with open('patch_dataset/labels/' + lab_name, 'w') as f:
        f.write(olab)

In [ ]:
out_img_dir = 'patch_dataset/images'
out_lab_dir = 'patch_dataset/labels'
out_imgs = os.listdir(out_img_dir)

for img_name in out_imgs:
    lab_name = os.path.splitext(img_name)[0] + '.txt'
    img_path = out_img_dir + '/' + img_name
    lab_path = out_lab_dir + '/' + lab_name
    lab = np.loadtxt(lab_path)
    if len(lab) == 0:
        continue
    lab = lab[:, 1:]
    img = Image.open(img_path).convert('RGB')
    w, h = img.size
    lab[:, 0] *= w
    lab[:, 2] *= w
    lab[:, 1] *= h
    lab[:, 3] *= h
    fig, ax = plt.subplots(1)
    plt.xticks([])
    plt.yticks([])
    ax.imshow(img)
    for box in lab:
        x, y, w, h = box
        x1, y1, x2, y2 = xywh2xyxy(box)
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()